# Install

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import math
import pandas as pd
from numpy import *
import torch
from scipy.stats import sem
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Function

In [ ]:
def read_csv(csv_path):
  url_data = csv_path.replace('/edit#gid=', '/export?format=csv&gid=')
  return pd.read_csv(url_data)

In [ ]:
def plot_model_weight(dict_weight, fig_name):
  df_weight = pd.DataFrame(dict_weight)
  
  plt.figure(figsize=(16, 4))

  df_weight.plot.bar(stacked=True)

  plt.ylabel("weights", fontweight='bold')
  plt.xticks(rotation=90)
  plt.legend().set_visible(True)
  plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.85),ncol=1)
  plt.grid(True, linestyle='-')
  plt.ylim(-2.5, 0)

  for label in df_weight.keys():
    for x, y in enumerate(df_weight[label]):
        plt.text(x, y, y, verticalalignment='center', horizontalalignment='center', va='top', visible=True)

  plt.savefig(fig_name, dpi=500, bbox_inches='tight')
  plt.show()

In [ ]:
def plot_model_weight_human_lm(dict_human, dict_bert, dict_gpt, fig_name):
  df_human = pd.DataFrame(dict_human)
  df_bert = pd.DataFrame(dict_bert)
  df_gpt = pd.DataFrame(dict_gpt)

  #plot 1: Human
  plt.figure(figsize=(16,4))  
  
  ax1 = plt.subplot(1,3,1) 

  df_human.plot.bar(stacked=True,ax=ax1)

  plt.ylabel("weights",fontweight='bold')
  plt.xticks(rotation=90)
  plt.legend().set_visible(False)
  plt.grid(True,linestyle = '-')
  plt.ylim(-2.5,0)
  for label in df_human.keys():
    for x,y in enumerate(df_human[label]):
      plt.text(x,y,y,verticalalignment='center',horizontalalignment='center',va= 'top', visible=True)

  plt.title("Human")

  #plot 2: BERT
  ax2 = plt.subplot(1,3,2)
  df_bert.plot.bar(stacked=True, ax=ax2)
  plt.xticks(rotation=90)
  plt.legend().set_visible(False)
  plt.grid(True,linestyle = '-')
  plt.ylim(-2.5,0)
  for label in df_bert.keys():
    for x,y in enumerate(df_bert[label]):
      plt.text(x,y,y,verticalalignment='center',horizontalalignment='center',va= 'top', visible=True)
  plt.title("BERT")

  #plot 3: GPT-2
  ax3 = plt.subplot(1,3,3)
  df_gpt.plot.bar(stacked=True, ax=ax3)
  plt.xticks(rotation=90)
  plt.legend(loc='center left', bbox_to_anchor=(1.1, 0.85),ncol=1)
  plt.grid(True,linestyle = '-')
  plt.ylim(-2.5,0)
  for label in df_gpt.keys():
    for x,y in enumerate(df_gpt[label]):
      plt.text(x,y,y,verticalalignment='center',horizontalalignment='center',va= 'top', visible=True)
  plt.title("GPT-2")

  plt.savefig(fig_name, dpi=500, bbox_inches='tight')
  plt.show()

In [ ]:
def plot_accep_lp(df_bert, df_gpt, filedr, nb_exp):

  # 1) Compute 95 confidence intervelle and 2) reset index and add as new columns
  df_accep_mean = df_bert.groupby(['Grammatical function', 'Extraction type']).acceptability_mean.mean().reset_index()
  df_accep_se = df_bert.groupby(['Grammatical function', 'Extraction type']).acceptability_mean.apply(sem).mul(1.96).reset_index()

  df_bert_mean = df_bert.groupby(['Grammatical function', 'Extraction type']).mean_lp.mean().reset_index()
  df_bert_se = df_bert.groupby(['Grammatical function', 'Extraction type']).mean_lp.apply(sem).mul(1.96).reset_index()

  df_gpt_mean = df_gpt.groupby(['Grammatical function', 'Extraction type']).mean_lp.mean().reset_index()
  df_gpt_se = df_gpt.groupby(['Grammatical function', 'Extraction type']).mean_lp.apply(sem).mul(1.96).reset_index()

  # Filter data for 'subject'
  df_accep_subject = df_accep_mean[df_accep_mean['Grammatical function'] == 'subject']
  df_accep_se_subject = df_accep_se[df_accep_se['Grammatical function'] == 'subject']
  df_bert_subject = df_bert_mean[df_bert_mean['Grammatical function'] == 'subject']
  df_bert_se_subject = df_bert_se[df_bert_se['Grammatical function'] == 'subject']
  df_gpt_subject = df_gpt_mean[df_gpt_mean['Grammatical function'] == 'subject']
  df_gpt_se_subject = df_gpt_se[df_gpt_se['Grammatical function'] == 'subject']

  # Filter data for 'object'
  df_accep_object = df_accep_mean[df_accep_mean['Grammatical function'] == 'object']
  df_accep_se_object = df_accep_se[df_accep_se['Grammatical function'] == 'object']
  df_bert_object = df_bert_mean[df_bert_mean['Grammatical function'] == 'object']
  df_bert_se_object = df_bert_se[df_bert_se['Grammatical function'] == 'object']
  df_gpt_object = df_gpt_mean[df_gpt_mean['Grammatical function'] == 'object']
  df_gpt_se_object = df_gpt_se[df_gpt_se['Grammatical function'] == 'object']

  # Sort based on 'Extraction type' column in alphabetical order
  df_accep_subject = df_accep_subject.sort_values('Extraction type')
  df_accep_se_subject = df_accep_se_subject.sort_values('Extraction type')
  df_accep_object = df_accep_object.sort_values('Extraction type')
  df_accep_se_object = df_accep_se_object.sort_values('Extraction type')

  df_bert_subject = df_bert_subject.sort_values('Extraction type')
  df_bert_se_subject = df_bert_se_subject.sort_values('Extraction type')
  df_bert_object = df_bert_object.sort_values('Extraction type')
  df_bert_se_object = df_bert_se_object.sort_values('Extraction type')

  df_gpt_subject = df_gpt_subject.sort_values('Extraction type')
  df_gpt_se_subject = df_gpt_se_subject.sort_values('Extraction type')
  df_gpt_object = df_gpt_object.sort_values('Extraction type')
  df_gpt_se_object = df_gpt_se_object.sort_values('Extraction type')


  # store
  df_merged_accep = pd.merge(df_accep_mean, df_accep_se, on=['Grammatical function', 'Extraction type'], suffixes=('_mean', '_se'))
  df_merged_bert = pd.merge(df_bert_mean, df_bert_se, on=['Grammatical function', 'Extraction type'], suffixes=('_mean', '_se'))
  df_merged_gpt = pd.merge(df_gpt_mean, df_gpt_se, on=['Grammatical function', 'Extraction type'], suffixes=('_mean', '_se'))

  df_merged = pd.concat([df_merged_accep, df_merged_bert, df_merged_gpt], keys=['accep', 'bert', 'gpt'])

  df_merged = df_merged.reset_index(level=0)
  df_merged.rename(columns={'level_0': 'Source'}, inplace=True)
  df_merged.to_csv(filedr+'exp'+str(nb_exp)+'.csv', index=False)

  # Create subplots
  fig, axes = plt.subplots(1, 3, figsize=(15, 5))

  # Plot mean acceptability
  axes[0].errorbar(df_accep_subject['Extraction type'], df_accep_subject['acceptability_mean'], yerr=df_accep_se_subject['acceptability_mean'], fmt='o-', capsize=4, label='subject', color='r')
  axes[0].errorbar(df_accep_object['Extraction type'], df_accep_object['acceptability_mean'], yerr=df_accep_se_object['acceptability_mean'], fmt='o-', capsize=4, label='object', color='b')
  axes[0].set_title('Human',fontweight='bold')
  #axes[0].set_xlabel('Extraction Type')
  axes[0].set_ylabel('Mean Acceptability')
  axes[0].set_ylim(0, 7)
  axes[0].tick_params(axis='x', rotation=90)
  #axes[0].legend()

  # Plot BERT mean lp
  axes[1].errorbar(df_bert_subject['Extraction type'], df_bert_subject['mean_lp'], yerr=df_bert_se_subject['mean_lp'], fmt='o-', capsize=4, label='subject', color='r')
  axes[1].errorbar(df_bert_object['Extraction type'], df_bert_object['mean_lp'], yerr=df_bert_se_object['mean_lp'], fmt='o-', capsize=4, label='object', color='b')
  axes[1].set_title('BERT',fontweight='bold')
  axes[1].set_xlabel('Extraction Types', fontsize=12)
  axes[1].set_ylabel('Mean Log-Probability')
  axes[1].set_ylim(-14, -7)
  axes[1].tick_params(axis='x', rotation=90)
  #axes[1].legend()

  # Plot GPT mean lp
  axes[2].errorbar(df_gpt_subject['Extraction type'], df_gpt_subject['mean_lp'], yerr=df_gpt_se_object['mean_lp'], fmt='o-', capsize=4, label='subject', color='r')
  axes[2].errorbar(df_gpt_object['Extraction type'], df_gpt_object['mean_lp'], yerr=df_gpt_se_object['mean_lp'], fmt='o-', capsize=4, label='object', color='b')
  axes[2].set_title('GPT-2',fontweight='bold')
  axes[2].tick_params(axis='x', rotation=90)
  axes[2].set_ylim(-7, 0)
  axes[2].set_ylabel('Mean Log-Probability')
  axes[2].legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

  # Adjust layout spacing
  plt.tight_layout()
  plt.subplots_adjust(wspace=0.18, hspace=0.3)
  plt.savefig('exp'+str(nb_exp)+'.png', dpi=500, bbox_inches='tight')
  # Show the plot
  plt.show()

In [ ]:
def analyse_position_6(df, condition = 'condition'):
  lst_mean_surprisal_1 = []
  lst_mean_surprisal_2 = []
  lst_mean_surprisal_3 = []
  lst_mean_surprisal_4 = []
  lst_mean_surprisal_5 = []
  lst_mean_surprisal_6 = []

  lst_sem_surprisal_1 = []
  lst_sem_surprisal_2 = []
  lst_sem_surprisal_3 = []
  lst_sem_surprisal_4 = []
  lst_sem_surprisal_5 = []
  lst_sem_surprisal_6 = []

  lst_y = []
  lst_y_err = []

  lst_condition = list(df[condition].value_counts().index)

  for conditon in lst_condition:
    df_condition = df[df[condition]==conditon]
    lst_mean_surprisal_1.append(df_condition['surprisal_zone1'].mean())
    lst_mean_surprisal_2.append(df_condition['surprisal_zone2'].mean())
    lst_mean_surprisal_3.append(df_condition['surprisal_zone3'].mean())
    lst_mean_surprisal_4.append(df_condition['surprisal_zone4'].mean())
    lst_mean_surprisal_5.append(df_condition['surprisal_zone5'].mean())
    lst_mean_surprisal_6.append(df_condition['surprisal_zone6'].mean())

    lst_sem_surprisal_1.append(sem(df_condition['surprisal_zone1']))
    lst_sem_surprisal_2.append(sem(df_condition['surprisal_zone2']))
    lst_sem_surprisal_3.append(sem(df_condition['surprisal_zone3']))
    lst_sem_surprisal_4.append(sem(df_condition['surprisal_zone4']))
    lst_sem_surprisal_5.append(sem(df_condition['surprisal_zone5']))
    lst_sem_surprisal_6.append(sem(df_condition['surprisal_zone6']))

    lst_y.append([df_condition['surprisal_zone1'].mean(), df_condition['surprisal_zone2'].mean(), df_condition['surprisal_zone3'].mean(), df_condition['surprisal_zone4'].mean(),
                  df_condition['surprisal_zone5'].mean(), df_condition['surprisal_zone6'].mean()])
    lst_y_err.append([sem(df_condition['surprisal_zone1']), sem(df_condition['surprisal_zone2']),sem(df_condition['surprisal_zone3']), sem(df_condition['surprisal_zone4']),
                      sem(df_condition['surprisal_zone5']), sem(df_condition['surprisal_zone6'])])
  
  df_result = {'condition':lst_condition, '1':lst_mean_surprisal_1, '2':lst_mean_surprisal_2, '3':lst_mean_surprisal_3, '4':lst_mean_surprisal_4, '5':lst_mean_surprisal_5,
               '6':lst_mean_surprisal_6, 
               'sem_surprisal_1':lst_sem_surprisal_1, 'sem_surprisal_2':lst_sem_surprisal_2,'sem_surprisal_3':lst_sem_surprisal_3,'sem_surprisal_4':lst_sem_surprisal_4, 
               'sem_surprisal_5':lst_sem_surprisal_5, 'sem_surprisal_6':lst_sem_surprisal_6,
               'lst_y':lst_y, 'lst_y_err':lst_y_err}
  df_result = pd.DataFrame(df_result)
  return df_result

In [ ]:
def analyse_position_8(df, condition = 'condition'):
  lst_mean_surprisal_1 = []
  lst_mean_surprisal_2 = []
  lst_mean_surprisal_3 = []
  lst_mean_surprisal_4 = []
  lst_mean_surprisal_5 = []
  lst_mean_surprisal_6 = []
  lst_mean_surprisal_7 = []
  lst_mean_surprisal_8 = []

  lst_sem_surprisal_1 = []
  lst_sem_surprisal_2 = []
  lst_sem_surprisal_3 = []
  lst_sem_surprisal_4 = []
  lst_sem_surprisal_5 = []
  lst_sem_surprisal_6 = []
  lst_sem_surprisal_7 = []
  lst_sem_surprisal_8 = []

  lst_y = []
  lst_y_err = []

  lst_condition = list(df[condition].value_counts().index)

  for conditon in lst_condition:
    df_condition = df[df[condition]==conditon]
    lst_mean_surprisal_1.append(df_condition['surprisal_zone1'].mean())
    lst_mean_surprisal_2.append(df_condition['surprisal_zone2'].mean())
    lst_mean_surprisal_3.append(df_condition['surprisal_zone3'].mean())
    lst_mean_surprisal_4.append(df_condition['surprisal_zone4'].mean())
    lst_mean_surprisal_5.append(df_condition['surprisal_zone5'].mean())
    lst_mean_surprisal_6.append(df_condition['surprisal_zone6'].mean())
    lst_mean_surprisal_7.append(df_condition['surprisal_zone7'].mean())
    lst_mean_surprisal_8.append(df_condition['surprisal_zone8'].mean())

    lst_sem_surprisal_1.append(sem(df_condition['surprisal_zone1']))
    lst_sem_surprisal_2.append(sem(df_condition['surprisal_zone2']))
    lst_sem_surprisal_3.append(sem(df_condition['surprisal_zone3']))
    lst_sem_surprisal_4.append(sem(df_condition['surprisal_zone4']))
    lst_sem_surprisal_5.append(sem(df_condition['surprisal_zone5']))
    lst_sem_surprisal_6.append(sem(df_condition['surprisal_zone6']))
    lst_sem_surprisal_7.append(sem(df_condition['surprisal_zone7']))
    lst_sem_surprisal_8.append(sem(df_condition['surprisal_zone8']))

    lst_y.append([df_condition['surprisal_zone1'].mean(), df_condition['surprisal_zone2'].mean(), df_condition['surprisal_zone3'].mean(), df_condition['surprisal_zone4'].mean(),
                  df_condition['surprisal_zone5'].mean(), df_condition['surprisal_zone6'].mean(), df_condition['surprisal_zone7'].mean(), df_condition['surprisal_zone8'].mean()])
    lst_y_err.append([sem(df_condition['surprisal_zone1']), sem(df_condition['surprisal_zone2']),sem(df_condition['surprisal_zone3']), sem(df_condition['surprisal_zone4']),
                      sem(df_condition['surprisal_zone5']), sem(df_condition['surprisal_zone6']),sem(df_condition['surprisal_zone7']), sem(df_condition['surprisal_zone8'])])
  
  df_result = {'condition':lst_condition, '1':lst_mean_surprisal_1, '2':lst_mean_surprisal_2, '3':lst_mean_surprisal_3, '4':lst_mean_surprisal_4, '5':lst_mean_surprisal_5,
               '6':lst_mean_surprisal_6, '7':lst_mean_surprisal_7, '8':lst_mean_surprisal_8, 
               'sem_surprisal_1':lst_sem_surprisal_1, 'sem_surprisal_2':lst_sem_surprisal_2,'sem_surprisal_3':lst_sem_surprisal_3,'sem_surprisal_4':lst_sem_surprisal_4, 
               'sem_surprisal_5':lst_sem_surprisal_5, 'sem_surprisal_6':lst_sem_surprisal_6,'sem_surprisal_7':lst_sem_surprisal_7,'sem_surprisal_8':lst_sem_surprisal_8,
               'lst_y':lst_y, 'lst_y_err':lst_y_err}
  df_result = pd.DataFrame(df_result)
  return df_result

In [ ]:
def plot_position_6(low_noetr, medium_noetr, high_noetr,low_pp, medium_pp, high_pp, fig_name):
  x = ['1', '2', '3', '4', '5', '6']
  fig, axes = plt.subplots(1, 2, figsize=(9, 4))
  axes[0].errorbar(x, eval(str(low_noetr['lst_y'].values[0])), yerr=eval(str(low_noetr['lst_y_err'].values[0])), label='low', color='#77AC30')
  axes[0].errorbar(x, eval(str(medium_noetr['lst_y'].values[0])), yerr=eval(str(medium_noetr['lst_y_err'].values[0])), label='medium', color='#D95319')
  axes[0].errorbar(x, eval(str(high_noetr['lst_y'].values[0])), yerr=eval(str(high_noetr['lst_y_err'].values[0])), label='high', color='#0072BD')
  #axes[0].legend(loc=1, prop={'size':6})
  axes[0].set_ylim(8.5,18.5)
  axes[0].set_ylabel('surprisal', fontsize=12)
  axes[0].set_title('no extraction',loc='left', fontsize=11, fontweight="bold")

  axes[1].errorbar(x, eval(str(low_pp['lst_y'].values[0])), yerr=eval(str(low_pp['lst_y_err'].values[0])), label='low', color='#77AC30')
  axes[1].errorbar(x, eval(str(medium_pp['lst_y'].values[0])), yerr=eval(str(medium_pp['lst_y_err'].values[0])), label='medium', color='#D95319')
  axes[1].errorbar(x, eval(str(high_pp['lst_y'].values[0])), yerr=eval(str(high_pp['lst_y_err'].values[0])), label='high', color='#0072BD')
  axes[1].legend(loc='center left', bbox_to_anchor=(1, 0.9),ncol=1, prop={'size':6})
  axes[1].set_ylim(8.5,18.5)
  #axes[1].set_ylabel('surprisal', fontsize=12)
  axes[1].set_title('PP-extracted',loc='left', fontsize=11, fontweight="bold")
  plt.savefig(fig_name+'.png', dpi=500, bbox_inches='tight')
  plt.show()

In [ ]:
def plot_position_8(noextr_subj, noextr_obj, pp_subj, pp_obj, missingde_subj, missingde_obj, fig_name):
  x = ['1', '2', '3', '4', '5', '6', '7', '8']
  plt.figure(figsize=(8, 4))
  plt.errorbar(x, eval(str(noextr_subj['lst_y'].values[0])), yerr=eval(str(noextr_subj['lst_y_err'].values[0])), label='no-extraction-subj', color='#77AC30')
  plt.errorbar(x, eval(str(noextr_obj['lst_y'].values[0])), yerr=eval(str(noextr_obj['lst_y_err'].values[0])), label='no-extraction-obj', color='#77AC30', linestyle='dotted')

  plt.errorbar(x, eval(str(pp_subj['lst_y'].values[0])), yerr=eval(str(pp_subj['lst_y_err'].values[0])), label='PP-extracted-subj', color='#D95319')
  plt.errorbar(x, eval(str(pp_obj['lst_y'].values[0])), yerr=eval(str(pp_obj['lst_y_err'].values[0])), label='PP-extracted-obj', color='#D95319', linestyle='dotted')

  plt.errorbar(x, eval(str(missingde_subj['lst_y'].values[0])), yerr=eval(str(missingde_subj['lst_y_err'].values[0])), label='ungrammm-subj', color='#0072BD')
  plt.errorbar(x, eval(str(missingde_obj['lst_y'].values[0])), yerr=eval(str(missingde_obj['lst_y_err'].values[0])), label='ungrammm-obj', color='#0072BD', linestyle='dotted')

  plt.legend(loc=1, prop={'size':6})
  plt.ylabel('surprisal', fontsize=12)
  plt.ylim(0,18)
  plt.savefig(fig_name+'.png', dpi=500, bbox_inches='tight')
  plt.show()

# Run

In [ ]:
DIR = ''

## Part 1

In [ ]:
df_en_pred_human_gap = pd.read_csv(DIR+'pred_human_gap.csv')
df_en_pred_bert_gap = pd.read_csv(DIR+'en_bert_pred_gap.csv')
df_en_pred_gpt = pd.read_csv(DIR+'en_gpt_pred_gap.csv')

df_fr_pred_human_gap = pd.read_csv(DIR+'pred_fr_human_gap.csv')
df_fr_pred_bert_gap = pd.read_csv(DIR+'pred_fr_bert_gap.csv')
df_fr_pred_gpt = pd.read_csv(DIR+'pred_fr_gpt.csv')

In [ ]:
dict_human_en_gap = {'Process':{'dist-filler-gap':0,'inter-syn-gap':-0.18, 'embed':-0.25},
               'Syntactic':{'extr-adj_hd':0,'extr-nom_hd':-0.28,'extr-no-subj':-0.2,'extr-subj-hd':-0.89},
               'Semantic':{'inter-rel-gap':-0.39,'no-pronoun-filler':0.05},
               'Discourse':{'focal-domain':-0.45}}
dict_bert_en_gap = {'Process':{'dist-filler-gap':0,'inter-syn-gap':-0.14, 'embed':-0.11},
               'Syntactic':{'extr-adj_hd':-0.26,'extr-nom_hd':0,'extr-no-subj':-0.07,'extr-subj-hd':-0.11},
               'Semantic':{'inter-rel-gap':0,'no-pronoun-filler':0},
               'Discourse':{'focal-domain':-0.38}}
dict_gpt_en_gap = {'Process':{'dist-filler-gap':0,'inter-syn-gap':0, 'embed':0},
               'Syntactic':{'extr-adj_hd':0,'extr-nom_hd':0,'extr-no-subj':-0.03,'extr-subj-hd':-0.08},
               'Semantic':{'inter-rel-gap':0,'no-pronoun-filler':0},
               'Discourse':{'focal-domain':-0.14}}

dict_human_fr_gap = {'Process':{'dist-filler-gap':-1.13,'inter-syn-gap':0, 'embed':0},
               'Syntactic':{'extr-adj_hd':0,'extr-nom_hd':0,'extr-no-subj':0,'extr-subj-hd':0},
               'Semantic':{'inter-rel-gap':0,'no-pronoun-filler':-0.69},
               'Discourse':{'focal-domain':-2}}
dict_bert_fr_gap = {'Process':{'dist-filler-gap':0,'inter-syn-gap':0, 'embed':-1.58},
               'Syntactic':{'extr-adj_hd':0,'extr-nom_hd':0,'extr-no-subj':0,'extr-subj-hd':0},
               'Semantic':{'inter-rel-gap':-0.44,'no-pronoun-filler':0},
               'Discourse':{'focal-domain':0}}
dict_gpt_fr_gap = {'Process':{'dist-filler-gap':0,'inter-syn-gap':0, 'embed':0},
               'Syntactic':{'extr-adj_hd': -0.85,'extr-nom_hd':0,'extr-no-subj':0,'extr-subj-hd':0},
               'Semantic':{'inter-rel-gap':0,'no-pronoun-filler':0},
               'Discourse':{'focal-domain':0}}

In [ ]:
plot_model_weight(dict_human_en_gap, 'en_human_gap.png')
plot_model_weight(dict_human_fr_gap, 'fr_human_gap.png')

plot_model_weight_human_lm(dict_human_en_gap, dict_bert_en_gap, dict_gpt_en_gap, 'en_human_lm_gap.png')
plot_model_weight_human_lm(dict_human_fr_gap, dict_bert_fr_gap, dict_gpt_fr_gap, 'fr_human_lm_gap.png')

## Part 2

In [ ]:
df_exp1_bert = pd.read_csv(DIR+'exp1-bert.csv')
df_exp2_bert = pd.read_csv(DIR+'exp2-bert.csv')
df_exp3_bert = pd.read_csv(DIR+'exp3-bert.csv')
df_exp41_bert = pd.read_csv(DIR+'exp4.1-bert.csv')
df_exp42_bert = pd.read_csv(DIR+'exp4.2-bert.csv')
df_exp43_bert = pd.read_csv(DIR+'exp4.3-bert.csv')
df_exp51_bert = pd.read_csv(DIR+'exp5.1-bert.csv')
df_exp52_bert = pd.read_csv(DIR+'exp5.2-bert.csv')
df_exp6_bert = pd.read_csv(DIR+'exp6-bert.csv')

df_exp1_gpt = pd.read_csv(DIR+'exp1-gpt.csv')
df_exp2_gpt = pd.read_csv(DIR+'exp2-gpt.csv')
df_exp3_gpt = pd.read_csv(DIR+'exp3-gpt.csv')
df_exp41_gpt = pd.read_csv(DIR+'exp4.1-gpt.csv')
df_exp42_gpt = pd.read_csv(DIR+'exp4.2-gpt.csv')
df_exp43_gpt = pd.read_csv(DIR+'exp4.3-gpt.csv')
df_exp51_gpt = pd.read_csv(DIR+'exp5.1-gpt.csv')
df_exp52_gpt = pd.read_csv(DIR+'exp5.2-gpt.csv')
df_exp6_gpt = pd.read_csv(DIR+'exp6-gpt.csv')

In [ ]:
plot_accep_lp(df_exp1_bert, df_exp1_gpt, DIR, 1)
plot_accep_lp(df_exp2_bert, df_exp2_gpt, DIR, 2)
plot_accep_lp(df_exp3_bert, df_exp3_gpt, DIR, 3)

plot_accep_lp(df_exp41_bert, df_exp41_gpt, DIR, 4.1)
plot_accep_lp(df_exp42_bert, df_exp42_gpt, DIR, 4.2)
plot_accep_lp(df_exp43_bert, df_exp43_gpt, DIR, 4.3)
plot_accep_lp(df_exp51_bert, df_exp51_gpt, DIR, 5.1)
plot_accep_lp(df_exp52_bert, df_exp52_gpt, DIR, 5.2)
plot_accep_lp(df_exp6_bert, df_exp6_gpt, DIR, 6)

## Part 3

###dont-rel

In [ ]:
exp7_bert = pd.read_csv(DIR+'exp7-bert.csv')
exp7_gpt = pd.read_csv(DIR+'exp7-gpt.csv')

In [ ]:
df_exp7_noextr_low_bert = df_exp7_bert_result[df_exp7_bert_result['condition']=='no-extraction-low']
df_exp7_noextr_medium_bert = df_exp7_bert_result[df_exp7_bert_result['condition']=='no-extraction-medium']
df_exp7_noextr_high_bert = df_exp7_bert_result[df_exp7_bert_result['condition']=='no-extraction-high']
df_exp7_ppextracted_low_bert = df_exp7_bert_result[df_exp7_bert_result['condition']=='PP-extracted-low']
df_exp7_ppextracted_medium_bert = df_exp7_bert_result[df_exp7_bert_result['condition']=='PP-extracted-medium']
df_exp7_ppextracted_high_bert = df_exp7_bert_result[df_exp7_bert_result['condition']=='PP-extracted-high']

df_exp7_noextr_low_gpt = df_exp7_gpt_result[df_exp7_gpt_result['condition']=='no-extraction-low']
df_exp7_noextr_medium_gpt = df_exp7_gpt_result[df_exp7_gpt_result['condition']=='no-extraction-medium']
df_exp7_noextr_high_gpt = df_exp7_gpt_result[df_exp7_gpt_result['condition']=='no-extraction-high']
df_exp7_ppextracted_low_gpt = df_exp7_gpt_result[df_exp7_gpt_result['condition']=='PP-extracted-low']
df_exp7_ppextracted_medium_gpt = df_exp7_gpt_result[df_exp7_gpt_result['condition']=='PP-extracted-medium']
df_exp7_ppextracted_high_gpt = df_exp7_gpt_result[df_exp7_gpt_result['condition']=='PP-extracted-high']

In [ ]:
df_exp7_bert_result.to_csv('exp7_bert.csv')
df_exp7_gpt_result.to_csv('exp7_gpt.csv')

In [ ]:
plot_position_6(df_exp7_noextr_low_bert, df_exp7_noextr_medium_bert, df_exp7_noextr_high_bert,
               df_exp7_ppextracted_low_bert, df_exp7_ppextracted_medium_bert, df_exp7_ppextracted_high_bert,'exp7_bert')
plot_position_6(df_exp7_noextr_low_gpt, df_exp7_noextr_medium_gpt, df_exp7_noextr_high_gpt,
               df_exp7_ppextracted_low_gpt, df_exp7_ppextracted_medium_gpt, df_exp7_ppextracted_high_gpt, 'exp7-gpt')

### dequi-rel

In [ ]:
df_exp8_bert = pd.read_csv(DIR+'exp8-bert.csv')
df_exp8_gpt = pd.read_csv(DIR+'exp8-gpt.csv')
df_exp8_bert = analyse_position_8(df_exp8_bert)
df_exp8_gpt = analyse_position_8(df_exp8_gpt)

df_exp8_bert.to_csv('exp8_bert.csv')
df_exp8_gpt.to_csv('exp8_gpt.csv')

In [ ]:
df_exp8_noextr_obj_bert = df_exp8_bert[df_exp8_bert['condition']=='no-extraction-obj']
df_exp8_noextr_subj_bert = df_exp8_bert[df_exp8_bert['condition']=='no-extraction-subj']
df_exp8_pp_obj_bert =  df_exp8_bert[df_exp8_bert['condition']=='PP-extracted-obj']
df_exp8_pp_subj_bert =  df_exp8_bert[df_exp8_bert['condition']=='PP-extracted-subj']
df_exp8_missingde_subj_bert =  df_exp8_bert[df_exp8_bert['condition']=='missingde-subj']
df_exp8_missingde_obj_bert =  df_exp8_bert[df_exp8_bert['condition']=='missingde-obj']

df_exp8_noextr_obj_gpt = df_exp8_gpt[df_exp8_gpt['condition']=='no-extraction-obj']
df_exp8_noextr_subj_gpt = df_exp8_gpt[df_exp8_gpt['condition']=='no-extraction-subj']
df_exp8_pp_obj_gpt =  df_exp8_gpt[df_exp8_gpt['condition']=='PP-extracted-obj']
df_exp8_pp_subj_gpt =  df_exp8_gpt[df_exp8_gpt['condition']=='PP-extracted-subj']
df_exp8_missingde_subj_gpt =  df_exp8_gpt[df_exp8_gpt['condition']=='missingde-subj']
df_exp8_missingde_obj_gpt =  df_exp8_gpt[df_exp8_gpt['condition']=='missingde-obj']

In [ ]:
plot_position_8(df_exp8_noextr_obj_bert, df_exp8_noextr_subj_bert, df_exp8_pp_obj_bert, df_exp8_pp_subj_bert, df_exp8_missingde_subj_bert, df_exp8_missingde_obj_bert, 'exp8_bert')
plot_position_8(df_exp8_noextr_obj_gpt, df_exp8_noextr_subj_gpt, df_exp8_pp_obj_gpt, df_exp8_pp_subj_gpt, df_exp8_missingde_subj_gpt, df_exp8_missingde_obj_gpt, 'exp8_gpt')